In [6]:
!pip install -q langchain langchain-community sentence-transformers faiss-cpu transformers PyPDF2 unstructured pdfminer.six pillow-heif pymupdf


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
import os
import logging
from typing import List
from IPython.display import display
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [8]:
from IPython.display import display
import os

pdf_path = "/kaggle/input/resume-data/Ashraf__Gen_AI_Engineer.docx"

if not os.path.exists(pdf_path):
    print("❌ PDF not found. Please upload it manually using the Jupyter file browser.")
else:
    print("✅ PDF found!")


✅ PDF found!


In [9]:
# def load_and_chunk(file_path: str, chunk_size: int = 500, chunk_overlap: int = 100):
#     from langchain.text_splitter import RecursiveCharacterTextSplitter

#     loader = PyPDFLoader(file_path)
#     documents = loader.load()

#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap
#     )

#     chunks = splitter.split_documents(documents)
#     return chunks

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_chunk(file_path):
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    return splitter.split_documents(documents)


docs = load_and_chunk(pdf_path)
print(f"✅ Loaded {len(docs)} chunks.")


✅ Loaded 49 chunks.


In [10]:
def embed_and_store(docs, index_name="index_store"):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.from_documents(docs, embeddings)
    db.save_local(index_name)
    return db

db = embed_and_store(docs)


In [11]:
def load_retriever(index_name="index_store"):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    return FAISS.load_local(
        folder_path=index_name,
        embeddings=embeddings,
        allow_dangerous_deserialization=True  # ✅ This enables safe loading
    ).as_retriever()
retriever = load_retriever()

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.2,
    device=0  # CPU
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)


Device set to use cuda:0


In [15]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)
print("✅ RAG pipeline ready.")


✅ RAG pipeline ready.


In [ ]:
while True:
    query = input("\nYou: ")
    if query.lower() in ['exit', 'quit']:
        break

    response = rag_chain.invoke({"query": query})
    print("🤖:", response["result"])



You:  Hi


🤖: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Ashraf Syed
ashraf274612@gmail.com | +16823343746 | Python-Generative AI
Engineer | linkedin.com/in/ashraf-syed-84567458
Ashraf Syed
ashraf274612@gmail.com | +16823343746 | Python-Generative AI
Engineer | linkedin.com/in/ashraf-syed-84567458
Professional Summary
Over 8 years of experience in IT, specializing in Fin ops, Machine
Learning, AI, NLP/NLU, Generative AI, and LLM (OpenAI)
Generative AI Engineer with extensive experience in AI-driven
chatbots, Fin ops reports, and predictive analytics.

Project 2: Matilda Knowledge base – LLM Chat bot: Conversational
chatbot development with a Llama foundation model and Retrieval-
Augmented Generation (RAG) architecture that can respond to user
questions using internal company documents (data stored in Word,
PDF, and Confluence pages) with a focus on providing in-depth
responses 


You:  Important highlights from the resume?


🤖: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

create proactive re-engagement campaigns, which leads to longer-
term retention.
Optimized marketing campaigns based on ROI predictions and

Security & Compliance
Data Security, PII Compliance, PHI Compliance, Regulatory
Compliance, Data Integrity, IP Protections
Employment History
Employer: CNET Global Solution Inc | Generative AI Engineer |

Skills
Skill Category
Skills
Generative AI & NLP
AutoGen, LangGraph,LangSmith, Crew Ai, Large Language Models
(LLMs), Portkey, Transformers, BERT, Prompt Engineering, Natural
Language Processing (NLP), Natural Language Understanding (NLU)
Machine Learning
Machine Learning, Deep Learning, Predictive Modeling, Random
Forest, XGBoost, K-Means Clustering, Time Series Analysis,
Regression Models, Fine-tuning LLMs, Word2Vec, TF-IDF
Cloud & DevOps

in campaign effectiveness while optimizin